# NYC subway turnstile entries for 2016
This code cleans NYC subway turnstile data to so a model can be trained on January-November statrion entry numbers and then predict December entries by day and by station. 

In [ ]:
import datetime as dt
import calendar as cal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from time import perf_counter

from IPython.display import display, HTML

import config as c

## Read in all ~9.5 million rows of NYC subway turnstile data for 2016
* This drops columns that are not necessary for the analysis

In [ ]:
dtypes = {'Entries': 'int64', 'Exits': 'int64'}
df= pd.read_csv('../intermediate data/Turnstile_Usage_Data__2016.csv', dtype=dtypes, low_memory=False)
df.columns = df.columns.str.strip().str.lower().str.replace('/', '')
df.drop(columns=['line name', 'division', 'description', 'exits'], inplace=True)
print(df.dtypes)
display(HTML(df.head(10).to_html()))

## Loop through all data, cleaning it by station, then aggregating it by station and date.
This allows a smaller amount of memory to be used to clean and aggregate the data, thus resulting in a much smaller output dataset

In [ ]:
stations = df.station.unique()
n=0
nTotal = len(stations)

tt0 = perf_counter()

# Loop through stations to process smaller chunks of data at a time
for s in stations:
    t0 = perf_counter()
    n+=1
    
    # Print out info on station name and number in this dataset
    print('\n{}/{} | {} '.format(n, nTotal, s), end='\n')
    
    # Get slice of dataframe for each station, s
    dfStation = df[df['station']==s].copy()  

    # Make turnstile column, concatenate ca, unit, and scp fields
    dfStation['turnstile'] = dfStation['ca'] + ' ' +  dfStation['unit'] + ' ' + dfStation['scp']
    
    # Make date column a datetime object
    dfStation['datetime'] = pd.to_datetime(dfStation['date']  + ' ' + dfStation['time'])
    dfStation['date'] = pd.to_datetime(dfStation['date'])

    # Sort data for following entry calculations
    dfStation.sort_values(by=['turnstile', 'station', 'datetime'], inplace=True)

    # Calculate time interval
    dfStation['interval'] = dfStation['datetime'] - dfStation['datetime'].shift(1)
    
    # Calculate entries per time period
    dfStation['enterDiff'] = dfStation['entries'] - dfStation['entries'].shift(1)
    
    # Calculate entries per minute
    dfStation['intMin'] = dfStation['interval'] / np.timedelta64(1, 'm') 
    dfStation['entriesPerMin'] = dfStation['enterDiff'] / dfStation['intMin']
    
    # Clean up the dataframe - drop unneeded columns
    dfStation.drop(columns=['ca', 'unit', 'scp', 'time', 'entries'], inplace=True)
    
    # Find first occurance of each turnstile
    dfStation['firstSCP'] = ~ (dfStation['turnstile'] == dfStation['turnstile'].shift(1))
    
    # Remove bad data first occurances of turnstiles and negative entry numbers, NaNs,
    # where entires per minute > 30 (because let's be honest, more than 1 person
    # entering every 2 seconds is outrageous)
    before = dfStation.shape[0]
    print('Records before cleaning: {}'.format(before))
    dfStation = dfStation[(dfStation['firstSCP'] == False) & 
                          (dfStation['enterDiff']>=0) & 
                          (dfStation['entriesPerMin']<=30) &
                          (dfStation['intMin'] <= 300)].dropna(subset=['enterDiff'])
    after = dfStation.shape[0]
    print('Records after cleaning:  {}'.format(after))
    print('Retained {}% of records'.format(round(after/before*100, 2)))

#     plt.hist(dfStation['entriesPerMin'], range=[0, 30], bins=60, label='Entries Per Minute')
#     plt.yscale('log', nonposy='clip')
#     plt.xlabel('Entries Per Minute During reportying time period')
#     plt.ylabel('Samples in Data Set')
#     plt.legend()
#     plt.show()
    
    # Remove fields used only to clean the data
    dfStation.drop(columns=['turnstile', 'datetime', 'interval', 'intMin', 'entriesPerMin', 'firstSCP'],
                  inplace=True)
    
    # Calculate daily entries per station
    dfStationDay = dfStation.groupby(['station', 'date']).sum().reset_index()
    dfStationDay.rename(columns={'enterDiff': 'totalEntries'}, inplace=True)
    #display(HTML(dfStationDay.head(10).to_html()))
    print('Rows: {} (Grouped by station and day)'.format(dfStationDay.shape[0]))
    
    
    # Add data grouped by station and day to new df
    try:
        dfCleaned = dfCleaned.append(dfStationDay)
    except: 
        dfCleaned = dfStationDay
        
    print('Duration: {} sec.'.format(round(perf_counter() - t0, 2)))
    
    if n%5 == 0:
        intTime = round(perf_counter() - tt0, 2)
        print('\nINTERMEDIATE Duration: {} sec.'.format(intTime))
        remainingTime = round(intTime / n * (nTotal - n), 2)
        print('Estimated time remaining: {} sec.'.format(remainingTime))
    
    # This if statment is used in testing the code, to process a smaller
    # amount of data, allowing for faster iteration of code updates
    #if n>=2: break

dfCleaned.sort_values(by=['date', 'station'], inplace=True)
#display(HTML(dfCleaned.head(10).to_html()))

print('TOTAL Duration: {} sec.'.format(round(perf_counter() - tt0, 2)))
print(dfCleaned.shape)
cleanedDataCSV = '../intermediate data/dfCleaned.csv'
dfCleaned.to_csv(cleanedDataCSV, index=False)
del dfCleaned

## Group entry data by day and plot it to see if it makes sense

In [ ]:
cleanedDataCSV = '../intermediate data/dfCleaned_bak.csv'

dfClean = pd.read_csv(cleanedDataCSV, parse_dates=['date'])
dfClean.drop(columns=['station'], inplace=True)

dfByDay = dfClean.groupby(['date']).sum().reset_index()
dfByDay.sort_values(by=['date'], inplace=True)

titleText = 'Total entries into NYC subway system by day'
fig, ax = plt.subplots(figsize=(20, 2))
dfByDay.plot.line(ax=ax, x='date', y='totalEntries', figsize=(20,2))
ax.text(0.0035,1.15,titleText, 
        ha='left', va='top',
        color='black',
        bbox=dict(facecolor='lightgray', alpha=1),
        transform = ax.transAxes)
plt.show()

## Add columns to identify days of the week - For training the model
* 1 = True, i.e. row is that day
* 0 = False, i.e. row is not that day

In [ ]:
dfByDay['weekday'] = dfByDay['date'].dt.day_name()
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for d in days:
    dfByDay[d] = 0
    dfByDay.loc[dfByDay['weekday'] == d, d] = 1

dfByDay.drop(columns=['weekday'], inplace=True)
display(HTML(dfByDay.head(10).to_html()))

## Split data into training and testing sets
* Training set = January through November
* Testing set = December

In [ ]:
dfTrain = dfByDay[dfByDay['date']< '2016-12-1'].copy()
dfTest = dfByDay[dfByDay['date']>= '2016-12-1'].copy()

dfTrain.set_index('date', inplace=True)
dfTest.set_index('date', inplace=True)

## Fit a linear regression model to training set

In [ ]:
from sklearn.linear_model import LinearRegression

trainDays = dfTrain.drop(columns=['totalEntries'])
trainEntries = dfTrain['totalEntries']

model = LinearRegression(fit_intercept = False)
model.fit(trainDays, trainEntries)
dfTrain['predicted'] = model.predict(trainDays)

display(HTML(dfTrain.head(5).to_html()))

## Plot the regression model on top of entry data to visually examine fit

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
ax.plot(dfTrain['totalEntries'], color='gray', alpha=1)
ax.plot(dfTrain['predicted'], color='red', alpha=0.5)
ax.set_ylabel('Ridership')
ax.set_xticks
plt.show()